<a href="https://colab.research.google.com/github/AbhijeetBeedikar/YUVAi-Waste-Segregation-System/blob/main/Initial_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RUNTIME: AROUND 30 MINS

### 50% accuracy with annotated 12 class data version (1)

### 60-70% accuracy with 12 class data initial version

In [ ]:
import os
import math
import numpy as np
from keras.applications import DenseNet121
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from sklearn.metrics import confusion_matrix

In [ ]:
batch_size = 128
dim = 224
epochs_1 = 20
epochs_2 = 20
initial_learning_rate = 1e-3
dropEvery = 4
drop_rate = 0.5
save_model_name = 'garbage_classifier.h5'

In [ ]:
# Constants & functions
def count_img(dir):
    nb = 0
    for subdirectory in os.listdir(dir):
        nb += len(os.listdir(os.path.join(dir, subdirectory)))
    return nb

def plot_cm(labels, predictions, categories):
    predictions = np.argmax(predictions, axis=1)
    cm = confusion_matrix(labels, predictions)
    make_confusion_matrix(cm, percent=False, categories=categories)

def lr_step_decay(epoch, lr, d=drop_rate, e=dropEvery):
    drop_rate = d
    epochs_drop = e
    return initial_learning_rate * math.pow(drop_rate, math.floor(epoch/epochs_drop))


In [ ]:
main_dir = '/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/'
class_mode = 'categorical'
classes = os.listdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/test')
train_dir = os.path.join(main_dir, 'train(1)')
validation_dir = os.path.join(main_dir, 'valid(1)')
test_dir = os.path.join(main_dir, 'test(1)')
steps_per_epoch = len(os.listdir(train_dir)) // batch_size+1
validation_steps = len(os.listdir(validation_dir)) // batch_size+1
test_steps = len(os.listdir(test_dir)) // batch_size+1

In [ ]:
import os
os.listdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/train/clothes').index('train - clothes5166.jpg')

4262

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory( #make seperate folders based on classes in train,test and valid folders and classify the pictures in these folders accordingly. split tensorflow version of roboflow data into seperate images based on annotations, classify into folders then apply the following traning method
    train_dir,
    target_size=(dim, dim),
    batch_size=batch_size,
    class_mode=class_mode,
    classes=classes)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(dim, dim),
    batch_size=batch_size,
    class_mode=class_mode,
    classes=classes,
    shuffle=False)
validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(dim, dim),
    batch_size=batch_size,
    class_mode=class_mode,
    classes=classes)

Found 12095 images belonging to 10 classes.
Found 1535 images belonging to 10 classes.
Found 1492 images belonging to 10 classes.


In [ ]:
conv_base = DenseNet121(weights='imagenet', include_top=False, input_shape=(dim, dim, 3))
conv_base.trainable = False
model = models.Sequential()
model.add(conv_base)
model.add(layers.GlobalAveragePooling2D(input_shape=(4, 4, 1024)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(classes), activation='softmax'))

model.compile(optimizer=optimizers.Adam(lr=initial_learning_rate), loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=epochs_1, batch_size=batch_size,
                    validation_data=validation_generator, validation_steps=validation_steps, callbacks=[LearningRateScheduler(lr_step_decay, verbose=1)])

29084464/29084464 [==============================] - 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)



Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/20
1/1 [==============================] - 161s 161s/step - loss: 3.5621 - acc: 0.0469 - val_loss: 2.5602 - val_acc: 0.0547 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/20
1/1 [==============================] - 142s 142s/step - loss: 3.4802 - acc: 0.1250 - val_loss: 2.4605 - val_acc: 0.1484 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/20
1/1 [==============================] - 139s 139s/step - loss: 2.7093 - acc: 0.1094 - val_loss: 2.2194 - val_acc: 0.2500 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.001.
Epoch 4/20
1/1 [==============================] - 99s 99s/step - loss: 2.4562 - acc: 0.2266 - val_loss: 2.3619 - val_acc: 0.2344 - lr: 0.0010

Epoch 5: LearningRateScheduler setting learning rate to 0.0005.
Epoch 5/20
1/1 [==============================] - 104s 104s/step - loss: 2.8539 - acc: 0.1875 - val_loss:

In [ ]:
# Fine-tuning : unfreezing some of the top layers in the base of the pretrained convnet
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if 'conv5_block16' in layer.name:
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

model.compile(optimizer=optimizers.Adam(lr=initial_learning_rate), loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=epochs_2, batch_size=batch_size,
                    validation_data=validation_generator, validation_steps=validation_steps, callbacks=[LearningRateScheduler(lr_step_decay, verbose=1)])
os.chdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned')
model.save('50% classification model.h5')


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/20
1/1 [==============================] - 85s 85s/step - loss: 2.4929 - acc: 0.2578 - val_loss: 1.6271 - val_acc: 0.4219 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/20
1/1 [==============================] - 86s 86s/step - loss: 2.4410 - acc: 0.2188 - val_loss: 1.6082 - val_acc: 0.4922 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/20
1/1 [==============================] - 87s 87s/step - loss: 1.9718 - acc: 0.3750 - val_loss: 1.4637 - val_acc: 0.5547 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.001.
Epoch 4/20
1/1 [==============================] - 77s 77s/step - loss: 2.0065 - acc: 0.3906 - val_loss: 1.4867 - val_acc: 0.5312 - lr: 0.0010

Epoch 5: LearningRateScheduler setting learning rate to 0.0005.
Epoch 5/20
1/1 [==============================] - 85s 85s/step - loss: 1.8052 - acc: 0.3906 - val_loss: 1.3427 

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_steps)
print('test accuracy : ', test_acc)

In [ ]:
test_predictions = model.predict(test_generator, steps=test_steps)
test_labels = test_generator.classes
plot_cm(test_labels, test_predictions, classes)
plt.show()

1/1 [==============================] - 32s 32s/step


ValueError: ignored

In [ ]:
import os
import cv2
from sklearn.model_selection import train_test_split
os.chdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification')
classes = ['biological','brown-glass','cardboard','clothes','green-glass','metal','paper','plastic','sanitary waste and toothbrushes','shoes','white-glass']
for a in classes:
  train,leftover = train_test_split(os.listdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/'+a),train_size = 0.8)
  test,valid = train_test_split(leftover,test_size = 0.5)
  for b in train:
    os.chdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/train/'+a)
    cv2.imwrite('train - '+b,cv2.imread('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/'+a+'/'+b))
  for b in test:
    os.chdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/test/'+a)
    cv2.imwrite('test - '+b,cv2.imread('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/'+a+'/'+b))
  for b in valid:
    os.chdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/valid/'+a)
    cv2.imwrite('valid - '+b,cv2.imread('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/'+a+'/'+b))



KeyboardInterrupt: ignored

In [ ]:
import os
a = os.listdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/train(1)/glass')
a.sort()
print(a[-1])
print(os.listdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/train/glass').index('train - white-glass72.jpg'))
print(len(os.listdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/train/glass')))

12192.jpg
481
1609


In [ ]:
a = len(os.listdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/train(1)/clothes'))
a

3184

In [ ]:
from sklearn.model_selection import train_test_split
import cv2
import os
train,leftover = train_test_split(os.listdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/'+'battery'),train_size = 0.8)
test,valid = train_test_split(leftover,test_size = 0.5)
print(test)
print(train)
print(valid)
for b in train:
  os.chdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/train/battery')
  cv2.imwrite('train - '+b,cv2.imread('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/battery/'+b))
for b in test:
  os.chdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/test/battery')
  cv2.imwrite('test - '+b,cv2.imread('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/battery/'+b))
for b in valid:
  os.chdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/valid/battery')
  cv2.imwrite('valid - '+b,cv2.imread('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/battery/'+b))

['battery609.jpg', 'battery237.jpg', 'battery12.jpg', 'battery555.jpg', 'battery114.jpg', 'battery155.jpg', 'battery111.jpg', 'battery388.jpg', 'battery354.jpg', 'battery281.jpg', 'battery148.jpg', 'battery190.jpg', 'battery143.jpg', 'battery552.jpg', 'battery607.jpg', 'battery468.jpg', 'battery351.jpg', 'battery18.jpg', 'battery339.jpg', 'battery213.jpg', 'battery511.jpg', 'battery467.jpg', 'battery577.jpg', 'battery271.jpg', 'battery266.jpg', 'battery37.jpg', 'battery553.jpg', 'battery320.jpg', 'battery273.jpg', 'battery465.jpg', 'battery356.jpg', 'battery502.jpg', 'battery261.jpg', 'battery422.jpg', 'battery248.jpg', 'battery115.jpg', 'battery445.jpg', 'battery374.jpg', 'battery602.jpg', 'battery542.jpg', 'battery130.jpg', 'battery461.jpg', 'battery51.jpg', 'battery181.jpg', 'battery102.jpg', 'battery503.jpg', 'battery41.jpg', 'battery383.jpg', 'battery407.jpg', 'battery16.jpg', 'battery147.jpg', 'battery564.jpg', 'battery554.jpg', 'battery215.jpg', 'battery456.jpg', 'battery562.jpg

In [ ]:
classes = ['battery','biological','brown-glass','cardboard','clothes','green-glass','metal','paper','plastic','sanitary waste and toothbrushes','shoes','white-glass']

for b in ['test','train','valid']:
  os.chdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification')
  os.mkdir(b)
  os.chdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/'+b)
  for a in classes:
    os.mkdir(a)

FileExistsError: ignored

In [ ]:
from google.colab import drive
import os
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

for c in os.listdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/valid/green-glass'):
  os.chdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/valid/glass')
  cv2.imwrite(c,cv2.imread('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/valid/green-glass/'+c))



In [ ]:
print(len(os.listdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/train/glass')),
      len(os.listdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/train/green-glass'))+
      len(os.listdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/train/white-glass'))+
      len(os.listdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/train/brown-glass')))

1608 1608


In [ ]:
import cv2
for a in ['test','train','valid']:
  os.rmdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/'+a+'/brown-glass')
  os.rmdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/'+a+'/green-glass')
  os.rmdir('/content/drive/MyDrive/12 groups garbage classification data - cleaned/garbage_classification/'+a+'/white-glass')

IsADirectoryError: ignored